## Iterates through a soundsep csv, opens wav files, saves biosound H5 files for further analysis.
## make sure to update your file folders before running this

In [2]:
# from scipy import integrate, optimize

import soundfile as sf
import numpy as np
from scipy.signal import spectrogram, butter, sosfilt, sosfiltfilt, correlate
from scipy.io.wavfile import read, write
import IPython.display as ipd
# import sys
from collections import Counter
import os
import pandas as pd
# import pandas as pd
import numpy as np
import matplotlib.colors as mcolors
import pickle
import csv
from soundsig.sound import BioSound 
from soundsig.sound import WavFile


# from ipywidgets import interact, fixed
# from ipywidgets import widgets
import matplotlib.pyplot as plt

def plot_a_spectrogram(w, fs:int=48000, title:str=spectrogram, nfft:int=2048):
    ff, tt, Sxx = spectrogram(w, fs=fs, nfft=nfft)
    max_ff = 10000 # only plot the first 10kHz
    ff_index = np.where(ff < max_ff)
    ff = ff[ff_index]
    Sxx = Sxx[ff_index]

    plt.figure(figsize=(10, 5))
    plt.pcolormesh(tt, ff[:max_ff], Sxx[:max_ff], cmap='plasma', shading='auto')
    # plt.pcolormesh(tt, ff[:145], Sxx[:145], cmap='plasma', shading='gouraud')
    plt.title(title)
    plt.xlabel('t (sec)')
    plt.ylabel('Frequency (Hz)')
    plt.grid()


In [3]:
base_folder = '/Users/williamwood/Documents/Data/Field Station Recordings/for_soundsep/14_38_03_18_07_2023/'
#'/Users/williamwood/Documents/Sound_Localization/for_soundsep/2023 maeve and jaewon/14_38_03_18_07_2023-data'
#'/Users/williamwood/Documents/Sound_Localization/for_soundsep/maeves data'#'/Users/williamwood/Documents/Sound_Localization/for_soundsep/14-38-03_18-07-2023' #7_18_23__09_29_44'
wave_files_folder = base_folder
#'/Users/williamwood/Documents/Sound_Localization/for_soundsep/14-38-03_18-07-2023'
# wave_files_folder = '/Users/williamwood/Documents/Sound_Localization/for_soundsep/7_18_23__09_29_44'
# Sound_Localization/for_soundsep/7_18_23__09_29_44'

file_name = os.path.join(base_folder,'_appdata','save','segments.csv')

print(file_name)
df = pd.read_csv(file_name, sep=',',header=0)        
print(f'Length of dataframe: {len(df)}')
print(f'Column names: {df.columns}')
print(f'Tags: {df["Tags"].unique()}')

/Users/williamwood/Documents/Data/Field Station Recordings/for_soundsep/14_38_03_18_07_2023/_appdata/save/segments.csv
Length of dataframe: 4344
Column names: Index(['Unnamed: 0.1', 'Unnamed: 0', 'SourceName', 'SourceChannel',
       'StartIndex', 'StopIndex', 'Tags', 'Coords'],
      dtype='object')
Tags: ['["1"]' '["6"]' '["2"]' '["5"]' '["0"]' '["4"]' '["3"]' '[]']


In [4]:
os.chdir(base_folder)
if not os.path.exists('spectrograms'):
    os.makedirs('spectrograms')
if not os.path.exists('csv'):
    os.makedirs('csv')

# This will be the output directory
if not os.path.exists('h5filesAmp'):
    os.makedirs('h5filesAmp')
    

In [12]:
# Read the csv file as a pandas data frame
soundsepPD = df
nsounds = len(soundsepPD)
print(nsounds)
wav_names = {
    'BroGre1354M':'audio_423212900Hz_14-38-03_18-07-2023.wav',
    'WhiWhi6607F':'audio_432736310Hz_14-38-04_18-07-2023.wav',
    'BluGre1214F':'audio_442360000Hz_14-38-00_18-07-2023.wav',
    'WhiWhi1541M':'audio_458185540Hz_14-37-58_18-07-2023.wav'
}
plotMe=True
normalize = True

# check the wav files to see if they have the same sampling rate
srs = list()
for file in wav_names.values():
    filename = os.path.join(wave_files_folder, file)
    track = sf.SoundFile(filename)
    srs.append(track.samplerate)
    print(f'{filename} has a sample rate of {track.samplerate}')
    track.close()
fs = srs[0]

4344
/Users/williamwood/Documents/Data/Field Station Recordings/for_soundsep/14_38_03_18_07_2023/audio_423212900Hz_14-38-03_18-07-2023.wav has a sample rate of 48000
/Users/williamwood/Documents/Data/Field Station Recordings/for_soundsep/14_38_03_18_07_2023/audio_432736310Hz_14-38-04_18-07-2023.wav has a sample rate of 48000
/Users/williamwood/Documents/Data/Field Station Recordings/for_soundsep/14_38_03_18_07_2023/audio_442360000Hz_14-38-00_18-07-2023.wav has a sample rate of 48000
/Users/williamwood/Documents/Data/Field Station Recordings/for_soundsep/14_38_03_18_07_2023/audio_458185540Hz_14-37-58_18-07-2023.wav has a sample rate of 48000


In [52]:
df

,Unnamed: 0.1,Unnamed: 0,SourceName,SourceChannel,StartIndex,StopIndex,Tags,Coords
0,0,0,BroGre1354M,0,705034,709502,"[""1""]","[8.153053283691406, 11.92204475402832, 5.47700..."
1,1,1,BluGre1214F,2,739121,741816,"[""6""]","[11.00963020324707, 12.903932571411133, 5.4287..."
2,2,2,BluGre1214F,2,773851,776369,"[""2""]","[11.17556381225586, 11.674291610717773, 3.4400..."
3,3,3,BluGre1214F,2,844891,848558,"[""6""]","[10.986668586730957, 12.001487731933594, 3.246..."
4,4,4,BroGre1354M,0,910549,913839,"[""5""]","[9.732298851013184, 11.947017669677734, 0.9106..."
...,...,...,...,...,...,...,...,...
4339,4339,4339,WhiWhi1541M,3,297302736,297305283,"[""5""]",NaN
4340,4340,4340,BluGre1214F,2,297324812,297329004,"[""6""]",NaN
4341,4341,4341,BluGre1214F,2,297367001,297371644,"[""6""]",NaN
4342,4342,4342,BluGre1214F,2,298070022,298074525,"[""6""]",NaN


In [36]:
# def return_voc(df_index):
#     segment = df.iloc[df_index]
#     file_name = wav_names.get(segment['SourceName']) # get the filename from the bird name
#     # print(f'opening data file:{os.path.join(base_folder, wav_names.get(segment['SourceName'] ) ) }')
#     fs, data_wav = read(os.path.join(base_folder, wav_names.get(segment['SourceName'])))
#     chunk = data_wav[segment['StartIndex']:segment['StopIndex']]
#     return chunk


def read_audio_section(filename, start_frame, stop_frame):
    track = sf.SoundFile(filename)

    can_seek = track.seekable() # True
    if not can_seek:
        raise ValueError("Not compatible with seeking")

    sr = track.samplerate
    frames_to_read = stop_frame - start_frame
    track.seek(start_frame)
    audio_section = track.read(frames_to_read)
    track.close()
    return audio_section, sr

In [37]:
for row_num in range(2):
    go = True
    print(f'row_num={row_num}, startindex={df.iloc[row_num]["StartIndex"]}, stopindex={df.iloc[row_num]["StopIndex"]}')
    dt = df.iloc[row_num]['StopIndex'] - df.iloc[row_num]['StartIndex']
 
    if dt<460: # if the syllable is less than 5ms, skip it 
        print(f'Syllable is too short (dt = {dt}), padding')
        # go = False
        # elif (dt < 460):
        ptadd = int((460 - dt)/2 + 1) # nwinlen requires a syllable of at least 460 samples to calculate biosound stuff, don't know why- Frederic had this at 250 before
        istart = df.iloc[row_num]['StartIndex']-ptadd
        if istart < 0 :
            print ('Skipping short syllable at very start of file')
            go = False
        iend = df.iloc[row_num]['StopIndex'] + ptadd
        # if (iend > len(soundIN.data) ):
        #     print ('Skipping short syllable at eend of file')
        #     go = False
    else:
        istart = df.iloc[row_num]['StartIndex']
        iend = df.iloc[row_num]['StopIndex']
    print(f'go={go} istart={istart} iend={iend} row_num={row_num}')
    # print(f'dt={dt}, syll_start= {int(istart/fs)}, syll_end={int(iend/fs)}')

    if go:
        # soundIN = WavFile(file_name=os.path.join(wave_files_folder, wav_names.get(df.iloc[row_num]['SourceName'])))
        # soundSYL = soundIN.data[istart:iend]
        # load just the segment of the audio file that we want
        soundSyl, sr = read_audio_section(os.path.join(wave_files_folder, wav_names.get(df.iloc[row_num]['SourceName'])), start_frame=istart, stop_frame=iend)
        
        birdname = df.iloc[row_num]['SourceName']
        if len(df.iloc[row_num]['Tags']) > 0:
            calltype = df.iloc[row_num]['Tags']
        else:
            calltype = 'unknown'
            
            
        # # Normalize if wanted- I don't want to
        # if normalize :
        #     maxAmp = np.abs(soundIN.data).max() 
        # else:
        #     maxAmp = 1.0
        
        # Create BioSound Object
        # myBioSound = BioSound(soundWave=soundSyl, fs=fs, emitter=birdname, calltype = calltype)

row_num=0, startindex=705034, stopindex=709502
go=True istart=705034 iend=709502 row_num=0
row_num=1, startindex=739121, stopindex=741816
go=True istart=739121 iend=741816 row_num=1


In [27]:
dt

2695

## TODO The cell below reloads one of 4 wav files every loop because they require so much memory. Figure out how to avoid this in the future (ubuntu desktop is probably sufficient)
## but lazy loading wav files makes the most sense. 

In [43]:
plotMe = False
# normalize = True
# for row_num in range(nsounds):
def make_h5_file(row_num, plotMe=False):
    go = True
    dt = df.iloc[row_num]['StopIndex'] - df.iloc[row_num]['StartIndex']
    if dt<460: # if the syllable is less than 5ms, skip it 
        print(f'Syllable is too short (dt = {dt}), padding')
        # go = False
        # elif (dt < 460):
        ptadd = int((460 - dt)/2 + 1) # nwinlen requires a syllable of at least 460 samples to calculate biosound stuff, don't know why- Frederic had this at 250 before
        istart = df.iloc[row_num]['StartIndex']-ptadd
        if istart < 0 :
            print ('Skipping short syllable at very start of file')
            go = False
        iend = df.iloc[row_num]['StopIndex'] + ptadd
        # if (iend > len(soundIN.data) ):
        #     print ('Skipping short syllable at eend of file')
        #     go = False
    else:
        istart = df.iloc[row_num]['StartIndex']
        iend = df.iloc[row_num]['StopIndex']
    print(f'go={go} istart={istart} iend={iend} row_num={row_num}')

    if go:
        # soundIN = WavFile(file_name=os.path.join(wave_files_folder, wav_names.get(df.iloc[row_num]['SourceName'])))
        # soundSYL = soundIN.data[istart:iend]
        # load just the segment of the audio file that we want
        # soundSyl = read_audio_section(os.path.join(wave_files_folder, wav_names.get(df.iloc[row_num]['SourceName'])), int(istart/fs), int(iend/fs))
        soundSyl, sr = read_audio_section(os.path.join(wave_files_folder, wav_names.get(df.iloc[row_num]['SourceName'])), start_frame=istart, stop_frame=iend)

        birdname = df.iloc[row_num]['SourceName']
        if len(df.iloc[row_num]['Tags']) > 0:
            calltype = df.iloc[row_num]['Tags']
        else:
            calltype = 'unknown'
            
            
        # # Normalize if wanted- I don't want to
        # if normalize :
        #     maxAmp = np.abs(soundIN.data).max() 
        # else:
        #     maxAmp = 1.0
        
        # Create BioSound Object
        myBioSound = BioSound(soundWave=soundSyl, fs=fs, emitter=birdname, calltype = calltype)
                
        # Calculate the spectrogram and the rms
        myBioSound.spectroCalc(spec_sample_rate=1000, freq_spacing = 100, min_freq=0, max_freq=10000)
        myBioSound.rms = myBioSound.sound.std() 
        
        # Calculate amplitude enveloppe
        myBioSound.ampenv(cutoff_freq = 200, amp_sample_rate = 1000)
        
        # Calculate the power spectrum
        myBioSound.spectrum(f_high=10000)
        
        # Calculate fundamental and related values.  These are the default values.
        # For the estimation of the fundamental, four methods are available: 
        # 'AC' - Peak of the auto-correlation function
        # 'ACA' - Peak of envelope of auto-correlation function 
        # 'Cep' - First peak in cepstrum 
        # 'Stack' - Fitting of harmonic stacks (default - works well for zebra finches)
        
        myBioSound.fundest(maxFund = 6000, minFund = 300, lowFc = 200, highFc = 10000, 
                            minSaliency = 0.5, debugFig = 0, 
                            minFormantFreq = 500, maxFormantBW = 500, windowFormant = 0.1,
                            method='AC')
        
        # Calculate the MPS
        myBioSound.mpsCalc(window=0.01, Norm = True)
                    
        # save the H5 file / biosuoound object
        myBioSound.saveh5(os.path.join(base_folder, 'h5filesAmp', 'bird_%s_call_%s_%s.h5' % (birdname, calltype, row_num)))

        print(f'myBioSound.fund = {myBioSound.fund}')
        if plotMe: 
            print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
            print('                Bird %s    Call Type %s' % (myBioSound.emitter, myBioSound.type))
            myBioSound.plot(DBNOISE=50, f_low=250, f_high=10000)  
            print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

In [46]:
print('done')

done


In [51]:
# from multiprocessing import Pool

# # run the h5 file creation in parallel

# def test(x):
#     print(x)
# with Pool(4) as p:
#     p.map(test, range(nsounds))


from joblib import Parallel, delayed

res = Parallel(n_jobs=5)(delayed(make_h5_file)(x) for x in range(nsounds))

go=True istart=844891 iend=848558 row_num=3
go=True istart=739121 iend=741816 row_num=1go=True istart=773851 iend=776369 row_num=2

go=True istart=705034 iend=709502 row_num=0
go=True istart=910549 iend=913839 row_num=4
myBioSound.fund = 523.8807942875263
go=True istart=915664 iend=917825 row_num=5
myBioSound.fund = 548.5750552565742
go=True istart=938688 iend=943405 row_num=6
myBioSound.fund = 557.333804196519
go=True istart=955388 iend=958960 row_num=7
myBioSound.fund = 554.539237391722
go=True istart=982338 iend=984912 row_num=8
myBioSound.fund = 560.2329749231304
go=True istart=1047560 iend=1050258 row_num=9
myBioSound.fund = 539.4620773776139
go=True istart=1169083 iend=1174072 row_num=10
myBioSound.fund = 538.3349558757258
go=True istart=1246128 iend=1249389 row_num=11
myBioSound.fund = 533.0555023652784
go=True istart=1303039 iend=1305757 row_num=12
myBioSound.fund = 508.57459034309437
go=True istart=1425245 iend=1428007 row_num=13
myBioSound.fund = 525.0694330030511
go=True ist

/opt/miniconda3/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:476: RuntimeWarning: Number of calls to function has reached maxfev = 2200.
  warnings.warn(errors[info][0], RuntimeWarning)


myBioSound.fund = 537.9378024840415
go=True istart=3489638 iend=3493997 row_num=18
myBioSound.fund = 524.8281490774871
go=True istart=3497899 iend=3501349 row_num=19
myBioSound.fund = 531.7598689858349
go=True istart=3556095 iend=3559832 row_num=20
myBioSound.fund = 521.2854925088353
go=True istart=3568290 iend=3571354 row_num=21
myBioSound.fund = 534.9399176994799
go=True istart=3861844 iend=3864094 row_num=22
myBioSound.fund = 534.7313059554364
go=True istart=3951647 iend=3954447 row_num=23
myBioSound.fund = 573.5263744051838
go=True istart=3980540 iend=3984210 row_num=24
myBioSound.fund = 571.7975346412778
go=True istart=4050255 iend=4054956 row_num=25
myBioSound.fund = 518.8453425197551
go=True istart=4064941 iend=4067385 row_num=26
myBioSound.fund = 540.611884599833
go=True istart=4123849 iend=4128369 row_num=27
myBioSound.fund = 511.58662239054973
go=True istart=4133753 iend=4139144 row_num=28
myBioSound.fund = 507.22322179200637
go=True istart=4148928 iend=4151357 row_num=29
myB

/opt/miniconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


myBioSound.fund = 551.4044141908054
go=True istart=4622335 iend=4625488 row_num=48
myBioSound.fund = []
go=True istart=4642683 iend=4646956 row_num=49
myBioSound.fund = 526.7590298106664
go=True istart=4687981 iend=4692842 row_num=51
myBioSound.fund = 553.5550745259881
go=True istart=4700093 iend=4703296 row_num=52
myBioSound.fund = 520.6640469575862
go=True istart=4740784 iend=4743210 row_num=53
go=True istart=4678954 iend=4681713 row_num=50
myBioSound.fund = 553.3567011511183
go=True istart=4773748 iend=4777752 row_num=54
myBioSound.fund = 519.5276194131936
go=True istart=4786320 iend=4789458 row_num=55
myBioSound.fund = 547.9566358710359
go=True istart=4838951 iend=4842034 row_num=56
myBioSound.fund = 505.4711726537506
go=True istart=4868964 iend=4874537 row_num=57


...

myBioSound.fund = 657.5972811589252
go=True istart=24620213 iend=24635132 row_num=427
myBioSound.fund = 557.2006441550578
go=True istart=25152155 iend=25155197 row_num=428
myBioSound.fund = 795.4961455682254
go=True istart=25159949 iend=25162990 row_num=429
myBioSound.fund = 550.1776953403903
go=True istart=25189198 iend=25194210 row_num=430
myBioSound.fund = []
go=True istart=25217056 iend=25220570 row_num=431
myBioSound.fund = 544.5628813576469
go=True istart=25227089 iend=25229998 row_num=432
myBioSound.fund = 570.0261532196396
go=True istart=25247791 iend=25253230 row_num=433
myBioSound.fund = 496.477045094738
go=True istart=25258656 iend=25261908 row_num=434
myBioSound.fund = 507.3829112020336
go=True istart=25272896 iend=25275839 row_num=435
myBioSound.fund = 548.6050552834696
go=True istart=25291240 iend=25295852 row_num=436
myBioSound.fund = []
go=True istart=25298688 iend=25302387 row_num=437
myBioSound.fund = 579.661333971029
go=True istart=25356057 iend=25359408 row_num=438


/opt/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


myBioSound.fund = 626.3908985270848
go=True istart=78220298 iend=78229244 row_num=1445
myBioSound.fund = 505.5214714568946
go=True istart=78241058 iend=78248312 row_num=1446
myBioSound.fund = 494.58229852736616
go=True istart=78257664 iend=78264977 row_num=1447
myBioSound.fund = 383.7714010127803
go=True istart=78267776 iend=78273178 row_num=1448
myBioSound.fund = 1221.1563340602488
go=True istart=78343522 iend=78349535 row_num=1449
myBioSound.fund = 485.53901821496936
go=True istart=78354211 iend=78358738 row_num=1450
myBioSound.fund = 592.8584274637561
go=True istart=78361299 iend=78368422 row_num=1451
myBioSound.fund = 585.6544057044233
go=True istart=78474140 iend=78478981 row_num=1452
myBioSound.fund = 527.8495960872558
go=True istart=78485862 iend=78489085 row_num=1453
myBioSound.fund = 537.5888147544503
go=True istart=78494558 iend=78497601 row_num=1454
myBioSound.fund = 528.2148831669816
go=True istart=78517778 iend=78561665 row_num=1455
myBioSound.fund = 511.0649848758786
go=T

/opt/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


myBioSound.fund = 581.8896068176929
go=True istart=136873395 iend=136881970 row_num=2385
myBioSound.fund = 470.26244289403775
go=True istart=136908365 iend=136912860 row_num=2386
myBioSound.fund = 519.8806576176967
go=True istart=138529732 iend=138534501 row_num=2387
myBioSound.fund = 561.2251683823682
go=True istart=138559485 iend=138562024 row_num=2388
myBioSound.fund = 544.3134489025084
myBioSound.fund = 559.5901480422451
go=True istart=138572758 iend=138577342 row_num=2389
go=True istart=138584662 iend=138589162 row_num=2390
myBioSound.fund = 542.9891089139177
go=True istart=138653875 iend=138658550 row_num=2391
myBioSound.fund = 542.5379910920267
go=True istart=138659415 iend=138662427 row_num=2392
myBioSound.fund = 756.4196942591091
go=True istart=138746501 iend=138749668 row_num=2393
myBioSound.fund = 511.77767439854284
go=True istart=138788867 iend=138793861 row_num=2394
myBioSound.fund = 514.0587573108461
go=True istart=138828177 iend=138832986 row_num=2395
myBioSound.fund = 5

In [53]:
print(f"last row is {row_num}, saved to: {os.path.join(base_folder, 'h5filesAmp')}")

last row is 31, saved to: /Users/williamwood/Documents/Data/Field Station Recordings/for_soundsep/14_38_03_18_07_2023/h5filesAmp


In [14]:
# Now that the H5 files are saved, we move on..
# except it isn't working.  I'm going to try to do it all in one loop


In [19]:
os.path.join(base_folder, 'h5filesAmp')

'/Users/williamwood/Documents/Data/Field Station Recordings/July_Recordings/14_38_03_18_07_2023_Maeves_data/h5filesAmp'